# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

21/11/20 19:12:23 WARN Utils: Your hostname, toscan-laptop resolves to a loopback address: 127.0.1.1; using 10.110.89.186 instead (on interface wlp1s0)
21/11/20 19:12:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/20 19:12:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
housing = spark.read.format('csv').option('header', True).option('sep', ',').option('inferSchema', True).load("data/housing.csv")

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [3]:
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [4]:
housing.count()

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [5]:
housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [6]:
housing.filter(housing['population'] > 10000).count()

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [7]:
housing.select(['housing_median_age', 'total_rooms', 'median_house_value', 'population']).describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [8]:
import pyspark.sql.functions as F

housing.select(F.max('housing_median_age'), F.min('total_rooms'), F.avg('median_house_value')).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [9]:
housing.groupBy('ocean_proximity').count().sort(F.desc('count')).show()


+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [10]:
housing.groupBy('ocean_proximity').agg(F.avg('median_house_value').alias('avg_value')).show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [11]:
housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) as avg_value FROM df GROUP BY ocean_proximity ORDER BY avg_value DESC").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|       NEAR BAY|259212.31179039303|
|     NEAR OCEAN|249433.97742663656|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [12]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler().setInputCols(['housing_median_age', 'total_rooms', 'median_house_value', 'population']).setOutputCol('features')
housingAttrs = va.transform(housing).select('features')
housingAttrs.show(5)

+--------------------+
|            features|
+--------------------+
|[41.0,880.0,45260...|
|[21.0,7099.0,3585...|
|[52.0,1467.0,3521...|
|[52.0,1274.0,3413...|
|[52.0,1627.0,3422...|
+--------------------+
only showing top 5 rows



In [13]:
from pyspark.ml.stat import Correlation

coeff = Correlation.corr(housingAttrs, 'features', 'pearson').collect()[0][0]
print(f"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 $DenseMatrix([[ 1.        , -0.3612622 ,  0.10562341, -0.29624424],
             [-0.3612622 ,  1.        ,  0.13415311,  0.85712597],
             [ 0.10562341,  0.13415311,  1.        , -0.02464968],
             [-0.29624424,  0.85712597, -0.02464968,  1.        ]])


/home/toscan/dev/KTH/SML-DL-Assignments/.venv/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [14]:
housingCol1 = housing.withColumn('rooms_per_household', housing['total_rooms'] / housing['households'])
housingCol2 = housingCol1.withColumn('bedrooms_per_room', housing['total_bedrooms'] / housing['total_rooms'])
housingExtra = housingCol2.withColumn('population_per_household', housing['population'] / housing['households'])

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [15]:
renamedHousing = housingExtra.withColumnRenamed('median_house_value', 'label')

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [16]:
# label columns
colLabel = "label"

# categorical columns
colCat = "ocean_proximity"

# numerical columns
colNum = [ c for c in renamedHousing.columns if c not in [colLabel, colCat] ]

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [17]:
renamedHousing.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in colNum]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-------------------+-----------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|rooms_per_household|bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-------------------+-----------------+------------------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                  0|              207|                       0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-------------------+-----------------+------------------------+



As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [18]:
from pyspark.ml.feature import Imputer

imputer = Imputer().setStrategy("median").setInputCols(['total_bedrooms', 'bedrooms_per_room']).setOutputCols(['total_bedrooms', 'bedrooms_per_room'])
imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [19]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

va = VectorAssembler().setInputCols(colNum).setOutputCol('unscaledFeatures')
featuredHousing = va.transform(imputedHousing)

scaler = StandardScaler().setInputCol('unscaledFeatures').setOutputCol('scaledFeatures').setWithMean(True).setWithStd(True)
scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)
# scaledHousing.rdd.map(lambda x: [float(y) for y in x['scaledFeatures']]).toDF([a for a in 'asdfghjklmn']).summary().show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|    unscaledFeatures|      scaledFeatures|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-1.3278030546902...|
|  -122.22|   37.86|              21.0|     7099.0|        1

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [20]:
renamedHousing.select(F.countDistinct('ocean_proximity')).show()

+-------------------------------+
|count(DISTINCT ocean_proximity)|
+-------------------------------+
|                              5|
+-------------------------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [21]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer().setInputCol(colCat).setOutputCol('ocean_proximity_index')
idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_index|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                  3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [22]:
indexer.fit(renamedHousing).labels

['<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND']

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [23]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder().setInputCol('ocean_proximity_index').setOutputCol('ocean_proximity_encoded')
ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_index|ocean_proximity_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+---------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                  3.0|          (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [24]:
from pyspark.ml import Pipeline, PipelineModel

numPipeline = Pipeline(stages=[imputer, va, scaler])
catPipeline = Pipeline(stages=[indexer, encoder])
pipeline = Pipeline().setStages([numPipeline, catPipeline])
newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|    unscaledFeatures|      scaledFeatures|ocean_proximity_index|ocean_proximity_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [25]:
va2 = VectorAssembler().setInputCols(['scaledFeatures', 'ocean_proximity_encoded']).setOutputCol('features')
dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-1.3278030546902...|452600.0|
|[-1.3228118684350...|358500.0|
|[-1.3327942409452...|352100.0|
|[-1.3377854272003...|341300.0|
|[-1.3377854272003...|342200.0|
+--------------------+--------+
only showing top 5 rows



---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [26]:
trainSet, testSet = dataset.randomSplit([0.8, 0.2], seed=12345)

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [27]:
from pyspark.ml.regression import LinearRegression

# train the model
# lr = LinearRegression(regParam=0.1, solver="normal", weightCol="weight", maxIter=10, featuresCol="features", labelCol="label", predictionCol="prediction")
lr = LinearRegression(regParam=0.1, featuresCol="features", labelCol="label", predictionCol="prediction")
lrModel = lr.fit(trainSet)
trainingSummary = lrModel.summary

print(f"Coefficients: {lrModel.coefficients}, Intercept: {lrModel.intercept}")
print(f"RMSE: {trainingSummary.rootMeanSquaredError}")

Coefficients: [-57786.986556215365,-58544.159437698865,13039.219125844193,6944.264705427141,2315.7416506188156,-45922.812292991126,41845.8689300983,77456.75166354905,6653.428675477162,16566.609839656558,612.3543996081354,-117202.5216413496,-150937.52760351344,-112195.05918247443,-122054.24744560121], Intercept: 334730.2346991016
RMSE: 68098.59232390606


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [28]:
from pyspark.ml.evaluation import RegressionEvaluator

# make predictions on the test data
predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error.
evaluator = RegressionEvaluator(predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|207333.07169653787|106700.0|[-2.3260403057099...|
| 198380.2162627444|128900.0|[-2.2960931881793...|
|224968.29445514386|116100.0|[-2.2960931881793...|
|155383.32850658853| 70500.0|[-2.2960931881793...|
|181233.16823472088| 85600.0|[-2.2911020019242...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67108.09738450353


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [29]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

dt = DecisionTreeRegressor(featuresCol="features", labelCol="label", predictionCol="prediction")

# train the model
dtModel = dt.fit(trainSet)

# make predictions on the test data
predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|  172815.184991274|106700.0|[-2.3260403057099...|
| 229327.4808080808|128900.0|[-2.2960931881793...|
| 229327.4808080808|116100.0|[-2.2960931881793...|
|132216.78321678322| 70500.0|[-2.2960931881793...|
|132216.78321678322| 85600.0|[-2.2911020019242...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 66251.3065136398


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [30]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(featuresCol="features", labelCol="label", predictionCol="prediction")

# train the model
rfModel = rf.fit(trainSet)

# make predictions on the test data
predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|190009.54719298644|106700.0|[-2.3260403057099...|
|194629.30459953673|128900.0|[-2.2960931881793...|
|220910.16025010025|116100.0|[-2.2960931881793...|
| 142276.6876134964| 70500.0|[-2.2960931881793...|
|  149186.951560682| 85600.0|[-2.2911020019242...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 63411.949237415196


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [31]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gb = GBTRegressor(featuresCol="features", labelCol="label", predictionCol="prediction")

# train the model
gbModel = gb.fit(trainSet)

# make predictions on the test data
predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|114582.26926656245|106700.0|[-2.3260403057099...|
|144738.53489749593|128900.0|[-2.2960931881793...|
| 157263.0572982507|116100.0|[-2.2960931881793...|
| 82094.20409409214| 70500.0|[-2.2960931881793...|
| 84263.29540364548| 85600.0|[-2.2911020019242...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 53871.63835568974


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [32]:
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator

paramGrid = ParamGridBuilder()\
    .addGrid(rf.numTrees, [1, 5, 10]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

evaluator = RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, parallelism=4, numFolds=3)
cvModel = cv.fit(trainSet)

predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

21/11/20 19:13:27 WARN DAGScheduler: Broadcasting large task binary with size 1064.3 KiB
21/11/20 19:13:29 WARN DAGScheduler: Broadcasting large task binary with size 1561.4 KiB
21/11/20 19:13:30 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
21/11/20 19:13:31 WARN DAGScheduler: Broadcasting large task binary with size 1240.8 KiB
21/11/20 19:13:32 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
21/11/20 19:13:32 WARN DAGScheduler: Broadcasting large task binary with size 1240.8 KiB
21/11/20 19:13:34 WARN DAGScheduler: Broadcasting large task binary with size 1927.4 KiB
21/11/20 19:13:34 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
21/11/20 19:13:35 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
21/11/20 19:13:37 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
21/11/20 19:13:38 WARN DAGScheduler: Broadcasting large task binary with size 5.4 MiB
21/11/20 19:13:40 WARN DAGScheduler: Br

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|          134170.0|106700.0|[-2.3260403057099...|
|143180.35714285713|128900.0|[-2.2960931881793...|
|242749.64285714287|116100.0|[-2.2960931881793...|
| 89433.68725868725| 70500.0|[-2.2960931881793...|
| 91159.71428571429| 85600.0|[-2.2911020019242...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 50959.45351399603


---
# 7. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

## 1) Load the data

In [33]:
cc = spark.read.format('csv').option('header', True).option('sep', ',').option('inferSchema', True).load("data/ccdefault.csv")
cc = cc.withColumnRenamed('DEFAULT', 'label')
cc.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- label: integer (nullable = true)

## 2) Analyses

Statistic summury for basic features

In [34]:
cc.select(["AGE","SEX","EDUCATION","MARRIAGE","label"]).describe().show()

+-------+-----------------+------------------+------------------+------------------+-------------------+
|summary|              AGE|               SEX|         EDUCATION|          MARRIAGE|              label|
+-------+-----------------+------------------+------------------+------------------+-------------------+
|  count|            30000|             30000|             30000|             30000|              30000|
|   mean|          35.4855|1.6037333333333332|1.8531333333333333|1.5518666666666667|             0.2212|
| stddev|9.217904068090155|0.4891291960902602|0.7903486597207269|0.5219696006132467|0.41506180569093254|
|    min|               21|                 1|                 0|                 0|                  0|
|    max|               79|                 2|                 6|                 3|                  1|
+-------+-----------------+------------------+------------------+------------------+-------------------+



Number of items in classes

In [35]:
cc.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 6636|
|    0|23364|
+-----+-----+



In [36]:
payCol = [ f'PAY_{i}' for i in [0,2,3,4,5,6] ]

stat = cc.select( [ F.mean(c) for c in payCol ] )\
        .withColumn('min', F.least(*[ f'avg({c})' for c in payCol ]))
minimum = stat.select('min').first()[0]
name_min = " "
for col in stat.columns:
    if stat.select(col).first()[0]==minimum and col!='min':
        name_min = col
print("The month with the most in time payments is the one corresponding to (see Number next to PAY):",name_min)

The month with the most in time payments is the one corresponding to (see Number next to PAY): avg(PAY_6)


Check for NULL values

In [37]:
cc.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in cc.columns]).show()

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|  0|        0|  0|        0|       0|  0|    0|    0|    0|    0|    0|    0|        0|        0|        0|        0|        0|        0|       0|       0|       0|       0|       0|       0|    0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+



## Names of columns

In [38]:
colNum ='LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6'.split(',')
colOH = ['EDUCATION', 'MARRIAGE']
colOHed = [ f"{c}_OH" for c in colOH]

## Pipeline

In [39]:
va = VectorAssembler().setInputCols(colNum).setOutputCol('unscaledFeatures')
scaler = StandardScaler().setInputCol('unscaledFeatures').setOutputCol('scaledFeatures').setWithMean(True).setWithStd(True)
encoder = OneHotEncoder().setInputCols(colOH).setOutputCols(colOHed)

pippo = Pipeline(stages=[va, scaler, encoder])

dataset = VectorAssembler().setInputCols(['scaledFeatures'] + colOHed).setOutputCol('features')\
            .transform(pippo.fit(cc).transform(cc))\
            .select('features', 'label')

trainSet, testSet = dataset.randomSplit([0.8, 0.2], seed=12345)

21/11/20 19:14:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 3) Train Models
We decided to test the three methods in their default hyperparameters in order to be fair with all of them. Random Forest performs better than any other, which is anticipated because it is an ensemble method of the bagging family which trains multiple decision trees on different subsets of the training set. Logistic regression comes second. Its performance is close to random forest’s performance. That is because we have a binary classification problem which is not complex and has a lot of numerical data. Decision tree comes last performing really poorly in its default settings (AUC~0.3). This is why we increased the maximum depth to help it learn from a more complex model. However, this could lead to overfitting. 

In [40]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier

models = [ m.fit(trainSet) for m in [
    LogisticRegression(featuresCol="features", labelCol="label", predictionCol="prediction"),
    DecisionTreeClassifier(featuresCol="features", labelCol="label", predictionCol="prediction", maxDepth=30),
    RandomForestClassifier(featuresCol="features", labelCol="label", predictionCol="prediction")
] ]
predictions = [ m.transform(testSet) for m in models ]


21/11/20 19:14:46 WARN DAGScheduler: Broadcasting large task binary with size 1005.7 KiB
21/11/20 19:14:46 WARN DAGScheduler: Broadcasting large task binary with size 1040.7 KiB
21/11/20 19:14:46 WARN DAGScheduler: Broadcasting large task binary with size 1068.8 KiB
21/11/20 19:14:46 WARN DAGScheduler: Broadcasting large task binary with size 1090.9 KiB
21/11/20 19:14:47 WARN DAGScheduler: Broadcasting large task binary with size 1109.2 KiB
21/11/20 19:14:47 WARN DAGScheduler: Broadcasting large task binary with size 1124.9 KiB
21/11/20 19:14:47 WARN DAGScheduler: Broadcasting large task binary with size 1135.9 KiB


## Results

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

for p in predictions:
    evaluator = BinaryClassificationEvaluator()

    acc = p.filter(p["prediction"] == p["label"]).count() / p.count()
    auc = evaluator.evaluate(p)

    print(f"Accuracy {acc}")
    print(f"The AUC is = {auc}")
    print("----")

Accuracy 0.8118147602683685
The AUC is = 0.7218825704350956
----
Accuracy 0.7337587956144657
The AUC is = 0.6374587842637311
----
Accuracy 0.8159057437407953
The AUC is = 0.7755222727822595
----


## 4) How to improve the results?
In order to achieve better results in all methods, we could tune their hyperparameters either manually or with the usage of Grid Search (paramGrid) or random search (randomGrid) which is most of the time more efficient.
We could do some feature engineering to improve the results. For example, we could either find and remove features that do not add new information to the dataset, or features that have the same behavior as other features. We can create new features based on existing ones, for example, with binning and interaction features, especially on the simplest model such as logistic regression or decision tree.
To handle imbalanced data, we can use technics of oversampling and undersampling or give weights to classes.